In [1]:
import os

import camelot
import pandas as pd
import tabula

In [2]:
os.chdir('/home/sean/Downloads/')
pdf = 'ClosingStatementBuyer.pdf'

In [11]:
tables = camelot.read_pdf(pdf) #can only export as file
#try reading in with csvreader?
tables.export('ClosingStatementBuyer.csv')

In [13]:
df2 = pd.read_csv('ClosingStatementBuyer-page-1-table-1.csv')

In [14]:
df2.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Description,Buyer,NaN,NaN
1,NaN,P.O.C.,Debit,Credit
2,"Deposits, Credits, Debits",NaN,NaN,NaN
3,Sale Price of Property,NaN,"$300,000.00",NaN
4,seller credits buyer,NaN,NaN,"$4,354.97"


In [21]:
df_original = tabula.read_pdf(pdf)
df_original.head()

,Unnamed: 0,Description,Unnamed: 2,Buyer,Unnamed: 4
0,NaN,NaN,P.O.C.,Debit,Credit
1,"Deposits, Credits, Debits",NaN,NaN,NaN,NaN
2,Sale Price of Property,NaN,NaN,"$300,000.00",NaN
3,seller credits buyer,NaN,NaN,NaN,"$4,354.97"
4,Deposit,NaN,NaN,NaN,"$1,500.00"


In [62]:
def fix_columns(df):
    columns = ['Description', 'Type', 'POC', 'Debit', 'Credit']
    df.columns = columns
    return df.drop(0)

    
df = fix_columns(df_original)
df.head()

,Description,Type,POC,Debit,Credit
1,"Deposits, Credits, Debits",NaN,NaN,NaN,NaN
2,Sale Price of Property,NaN,NaN,"$300,000.00",NaN
3,seller credits buyer,NaN,NaN,NaN,"$4,354.97"
4,Deposit,NaN,NaN,NaN,"$1,500.00"
5,Lender Credit from Better Mortgage Corporation,NaN,NaN,NaN,"$1,318.00"


In [63]:
def get_headings(df):
    new_rows = pd.DataFrame()
    headings = []
    for index, row in df.iterrows():
        #print(row[1])
        if (pd.isnull(row['POC']) and 
            pd.isnull(row['Debit']) and
            pd.isnull(row['Credit'])):
            #print(row[0])
            headings.append(row[0])
            
    return headings

headings = get_headings(df)
headings.pop(4)
headings.pop(4)
headings

['Deposits, Credits, Debits',
 'Prorations',
 'New Loans',
 'Title Charges',
 'Government Recording and Transfer Charges']

In [64]:
def headings_as_type(df, headings):
    new_rows = pd.DataFrame()
    heading = 'blank'
    for index, row in df.iterrows():
        #print(row[1])
        if row[0] in headings:
            #print(row[0])
            heading = row[0]
        elif ('$' in str(row[2]) or
              '$' in str(row[3]) or
              '$' in str(row[4])):
            row['Type'] = heading
            new_rows = new_rows.append(row)
            #print(row[0])
    return new_rows

df = headings_as_type(df, headings)
df = df.set_index(df['Description']).drop('Description', axis=1)
df = df.fillna(0.0)
converted = df[df.columns[:3]].replace('[\$,]', '', regex=True).astype(float)
converted['Type'] = df['Type']
df = converted

In [65]:
df.loc['Subtotals', 'Type'] = 'Subtotals'
df.loc['Due From Buyer', 'Type'] = 'Due From Buyer'
df.loc['Totals', 'Type'] = 'Totals' 

In [67]:
groups = df.groupby('Type').sum()
groups

,Credit,Debit,POC
Type,,,
"Deposits, Credits, Debits",7172.97,300000.00,0.0
Due From Buyer,7106.53,0.00,0.0
Government Recording and Transfer Charges,0.00,354.00,0.0
New Loans,291152.49,3317.59,550.0
Prorations,17.99,174.52,0.0
Subtotals,298343.45,305449.98,550.0
Title Charges,0.00,1603.87,0.0
Totals,305449.98,305449.98,550.0


In [68]:
due = df.loc['Subtotals', 'Debit'] - df.loc['Subtotals', 'Credit'] 

In [102]:
expected = 9000 - 1500 - 550 -17.99
diff = due - expected
diff

174.5199999999695

In [114]:
df.loc[df['Type'] == 'Prorations', 'Debit'].sum()

174.52

In [100]:
expected_credit = (groups.loc[['New Loans', 'Title Charges'], 'Debit'].sum()
                   + groups.loc[['Government Recording and Transfer Charges'], 'Debit'].sum()
                   - groups.loc[['New Loans', 'Title Charges'], 'Credit'].sum()
                   + 291000
                   - 1318 
                   + 550
                  )
expected_credit

4354.97000000003

In [115]:
df.loc['seller credits buyer', 'Credit']


4354.97